# JAX arrays (jax.Array)

In [1]:
# create array
import jax
import jax.numpy as jnp

x = jnp.arange(5)
isinstance(x, jax.Array)  # True

True

In [2]:
# Array devices and sharding
x.devices()

{CpuDevice(id=0)}

In [3]:
x.sharding

SingleDeviceSharding(device=CpuDevice(id=0))

## Transformations

除了对数组进行操作的函数外，JAX 还包括许多对 JAX 函数进行操作的转换。这些转换包括
- jax.jit(): Just-in-time (JIT) compilation
- jax.vmap(): Vectorizing transform;
- jax.grad(): Gradient transform; 

In [ ]:
def selu(x, alpha=1.67, lambda_=1.05):
    return lambda_ * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

print(jax.jit(selu)(1.0))

1.05


In [8]:
# 也可以使用修饰器进行转换，这样就不要额外写 jax.jit(selu)
@jax.jit
def selu(x, alpha=1.67, lambda_=1.05):
    return lambda_ * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)


## Tracing

transformations 背后的神奇之处在于 Tracer 的概念。Tracers 是数组对象的抽象替代品，它被传递给JAX函数，以便提取函数编码的操作序列。

In [ ]:
@jax.jit
def f(x):
    print(x) # 传入的 x 不是一个 Array，而是一个Tracer对象
    return x + 1

x = jnp.arange(5)
result = f(x)

Traced<ShapedArray(int32[5])>with<DynamicJaxprTrace(level=1/0)>


打印的值不是数组x，而是表示x的基本属性的Tracer实例，比如它的形状和dtype。通过使用跟踪值执行函数，JAX可以在实际执行这些操作之前确定函数编码的操作序列：jit（）、vmap（）和grad（）等转换可以将这个输入操作序列映射到转换后的操作序列。

## Jaxprs

解析函数的底层逻辑，有点debug的感觉

In [14]:
def selu(x, alpha=1.67, lambda_=1.05):
    return lambda_ * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

In [16]:
x = jnp.arange(5.0)
jax.make_jaxpr(selu)(x) # 函数的执行逻辑

{ lambda ; a:f32[5]. let
    b:bool[5] = gt a 0.0
    c:f32[5] = exp a
    d:f32[5] = mul 1.6699999570846558 c
    e:f32[5] = sub d 1.6699999570846558
    f:f32[5] = pjit[
      name=_where
      jaxpr={ lambda ; g:bool[5] h:f32[5] i:f32[5]. let
          j:f32[5] = select_n g i h
        in (j,) }
    ] b a e
    k:f32[5] = mul 1.0499999523162842 f
  in (k,) }

## Pytrees

JAX函数和转换基本上是在数组上操作的，但在实践中，编写处理数组集合的代码是很方便的：例如，神经网络可能会将其参数组织在具有有意义键的数组字典中。JAX不是逐个处理这些结构，而是依赖于pytree抽象以统一的方式处理这些集合。

In [23]:
# (nested) list of parameters
params = [1, 2, (jnp.arange(3), jnp.ones(2))]

# print(params[2])

print(jax.tree.structure(params)) # structure 表示树的形状
print(jax.tree.leaves(params)) # leaves 是最底层的数组/标量值（忽略结构）

PyTreeDef([*, *, (*, *)])
[1, 2, Array([0, 1, 2], dtype=int32), Array([1., 1.], dtype=float32)]


In [24]:
# Dictionary of parameters
params = {'n': 5, 'W': jnp.ones((2, 2)), 'b': jnp.zeros(2)}

print(jax.tree.structure(params))
print(jax.tree.leaves(params))

PyTreeDef({'W': *, 'b': *, 'n': *})
[Array([[1., 1.],
       [1., 1.]], dtype=float32), Array([0., 0.], dtype=float32), 5]


In [26]:
# Named tuple of parameters
from typing import NamedTuple

class aaa(NamedTuple):
    a: int
    b: float

params = aaa(1, 5.0)
print(jax.tree.structure(params))
print(jax.tree.leaves(params))

PyTreeDef(CustomNode(namedtuple[aaa], [*, *]))
[1, 5.0]


## Pseudorandom numbers

JAX努力与NumPy兼容，但伪随机数生成是一个明显的例外。NumPy支持基于全局状态生成伪随机数的方法，该方法可以使用NumPy .random.seed（）进行设置。全局随机状态很难与JAX的计算模型进行交互，并且很难在不同的线程、进程和设备之间强制执行再现性。相反，JAX通过一个随机密钥显式地跟踪状态：

np.random.seed(42)

- 使用的是全局状态（global state）
- 在多线程/并行环境中容易产生不可控的随机行为


In [27]:
from jax import random

key = random.key(43)
print(key)

Array((), dtype=key<fry>) overlaying:
[ 0 43]


In [29]:
# 重复使用 key 会得到相同的随机结果
print(random.normal(key))
print(random.normal(key))

0.81039715
0.81039715


重要原则：不要重复使用同一个 key，除非你想得到相同的结果

如何生成不同的样本？使用 random.split(key)


In [30]:
new_key, subkey = random.split(key)

- split 会把一个 key 分裂成两个独立的 key
- 原 key 被“消耗”，不再使用
- subkey 用来生成随机值，new_key 用来生成下一个 key

In [33]:
for i in range(3):
    new_key, subkey = random.split(key)
    del key # The old key is consumed by split() -- we must never use it again.
    val = random.normal(subkey)
    del subkey  # The subkey is consumed by normal().
    print(f"draw {i}: {val}")
    key = new_key  # new_key is safe to use in the next iteration.

draw 0: 1.0474061965942383
draw 1: 1.0435152053833008
draw 2: -0.21346548199653625
